# ソフトマックス関数と損失関数

このノートブックでは、ソフトマックス関数と多クラス分類の損失関数について詳しく学習します。

## 学習目標
- ソフトマックス関数の数学的性質
- 交差エントロピー損失の導出
- 勾配の計算
- 数値的安定性の考慮


In [ ]:
# 必要なライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import softmax as scipy_softmax
import warnings
warnings.filterwarnings('ignore')

# 日本語フォントの設定
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")


## 1. ソフトマックス関数の実装と比較


In [ ]:
def softmax_naive(z):
    """
    ナイーブなソフトマックス関数の実装（数値的不安定）
    
    Parameters:
    z: 線形結合の結果
    
    Returns:
    softmax(z): ソフトマックス関数の値
    """
    exp_z = np.exp(z)
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

def softmax_stable(z):
    """
    数値的に安定なソフトマックス関数の実装
    
    Parameters:
    z: 線形結合の結果
    
    Returns:
    softmax(z): ソフトマックス関数の値
    """
    # 数値的安定性のため、最大値を引く
    z_shifted = z - np.max(z, axis=1, keepdims=True)
    exp_z = np.exp(z_shifted)
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

# テストケース
z_test = np.array([[1, 2, 3], [0, 0, 0], [-1, 0, 1], [10, 20, 30]])

print("=== ソフトマックス関数の比較 ===")
print(f"入力 z: {z_test}")

# 各実装の結果
naive_result = softmax_naive(z_test)
stable_result = softmax_stable(z_test)
scipy_result = scipy_softmax(z_test, axis=1)

print(f"\nナイーブ実装: {naive_result}")
print(f"安定実装: {stable_result}")
print(f"SciPy実装: {scipy_result}")

print(f"\n各行の合計:")
print(f"ナイーブ: {naive_result.sum(axis=1)}")
print(f"安定: {stable_result.sum(axis=1)}")
print(f"SciPy: {scipy_result.sum(axis=1)}")

# 数値的不安定性の例
z_large = np.array([[100, 101, 102]])
print(f"\n=== 数値的不安定性の例 ===")
print(f"大きな値の入力: {z_large}")

try:
    naive_large = softmax_naive(z_large)
    print(f"ナイーブ実装: {naive_large}")
except:
    print("ナイーブ実装: オーバーフローエラー")

stable_large = softmax_stable(z_large)
scipy_large = scipy_softmax(z_large, axis=1)
print(f"安定実装: {stable_large}")
print(f"SciPy実装: {scipy_large}")


## 2. ソフトマックス関数の可視化


In [ ]:
# ソフトマックス関数の可視化
z_range = np.linspace(-10, 10, 100)
z_2d = np.column_stack([z_range, np.zeros_like(z_range), -z_range])
softmax_2d = softmax_stable(z_2d)

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(z_range, softmax_2d[:, 0], 'r-', label='Class 0', linewidth=2)
plt.plot(z_range, softmax_2d[:, 1], 'g-', label='Class 1', linewidth=2)
plt.plot(z_range, softmax_2d[:, 2], 'b-', label='Class 2', linewidth=2)
plt.xlabel('z[0]')
plt.ylabel('Softmax Probability')
plt.title('Softmax Function for 3 Classes')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
plt.plot(z_range, softmax_2d.sum(axis=1), 'k-', linewidth=2)
plt.xlabel('z[0]')
plt.ylabel('Sum of Probabilities')
plt.title('Sum of Softmax Probabilities (should be 1)')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
# 3D可視化の準備
z1_range = np.linspace(-5, 5, 50)
z2_range = np.linspace(-5, 5, 50)
Z1, Z2 = np.meshgrid(z1_range, z2_range)
Z3 = np.zeros_like(Z1)

z_3d = np.stack([Z1.ravel(), Z2.ravel(), Z3.ravel()], axis=1)
softmax_3d = softmax_stable(z_3d)
softmax_3d = softmax_3d.reshape(Z1.shape + (3,))

plt.contourf(Z1, Z2, softmax_3d[:, :, 0], levels=20, alpha=0.8, cmap='Reds')
plt.colorbar(label='Class 0 Probability')
plt.xlabel('z[0]')
plt.ylabel('z[1]')
plt.title('Class 0 Probability (z[2] = 0)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 3. 交差エントロピー損失の実装


In [ ]:
def cross_entropy_loss(y_true, y_pred):
    """
    交差エントロピー損失の計算
    
    Parameters:
    y_true: 実際のラベル (n_samples,)
    y_pred: 予測確率 (n_samples, n_classes)
    
    Returns:
    loss: 交差エントロピー損失
    """
    n_samples = y_true.shape[0]
    n_classes = y_pred.shape[1]
    
    # ワンホットエンコーディング
    y_one_hot = np.eye(n_classes)[y_true]
    
    # 数値的安定性のため、確率をクリップ
    y_pred_clipped = np.clip(y_pred, 1e-15, 1 - 1e-15)
    
    # 交差エントロピー損失の計算
    loss = -np.mean(np.sum(y_one_hot * np.log(y_pred_clipped), axis=1))
    return loss

def cross_entropy_loss_naive(y_true, y_pred):
    """
    ナイーブな交差エントロピー損失の計算（数値的不安定）
    
    Parameters:
    y_true: 実際のラベル (n_samples,)
    y_pred: 予測確率 (n_samples, n_classes)
    
    Returns:
    loss: 交差エントロピー損失
    """
    n_samples = y_true.shape[0]
    n_classes = y_pred.shape[1]
    
    # ワンホットエンコーディング
    y_one_hot = np.eye(n_classes)[y_true]
    
    # 交差エントロピー損失の計算（クリップなし）
    loss = -np.mean(np.sum(y_one_hot * np.log(y_pred), axis=1))
    return loss

# テストケース
y_true_test = np.array([0, 1, 2, 0])
y_pred_test = np.array([
    [0.7, 0.2, 0.1],
    [0.1, 0.8, 0.1],
    [0.2, 0.3, 0.5],
    [0.6, 0.3, 0.1]
])

print("=== 交差エントロピー損失の計算 ===")
print(f"実際のラベル: {y_true_test}")
print(f"予測確率: {y_pred_test}")

loss_stable = cross_entropy_loss(y_true_test, y_pred_test)
loss_naive = cross_entropy_loss_naive(y_true_test, y_pred_test)

print(f"\n安定実装の損失: {loss_stable:.4f}")
print(f"ナイーブ実装の損失: {loss_naive:.4f}")

# 極端なケースでの比較
y_pred_extreme = np.array([
    [0.99, 0.005, 0.005],
    [0.005, 0.99, 0.005],
    [0.005, 0.005, 0.99],
    [0.99, 0.005, 0.005]
])

print(f"\n=== 極端なケースでの比較 ===")
print(f"極端な予測確率: {y_pred_extreme}")

loss_stable_extreme = cross_entropy_loss(y_true_test, y_pred_extreme)
loss_naive_extreme = cross_entropy_loss_naive(y_true_test, y_pred_extreme)

print(f"安定実装の損失: {loss_stable_extreme:.4f}")
print(f"ナイーブ実装の損失: {loss_naive_extreme:.4f}")

# 損失関数の可視化
y_true_viz = np.array([0, 1, 2, 0])
y_pred_viz = np.linspace(0.01, 0.99, 100)

losses = []
for pred in y_pred_viz:
    y_pred_test_viz = np.array([
        [pred, (1-pred)/2, (1-pred)/2],
        [(1-pred)/2, pred, (1-pred)/2],
        [(1-pred)/2, (1-pred)/2, pred],
        [pred, (1-pred)/2, (1-pred)/2]
    ])
    loss = cross_entropy_loss(y_true_viz, y_pred_test_viz)
    losses.append(loss)

plt.figure(figsize=(10, 6))
plt.plot(y_pred_viz, losses, 'b-', linewidth=2)
plt.xlabel('Predicted Probability')
plt.ylabel('Cross-Entropy Loss')
plt.title('Cross-Entropy Loss vs Predicted Probability')
plt.grid(True, alpha=0.3)
plt.show()


## 4. 勾配の計算


In [ ]:
def compute_gradients(X, y_true, y_pred):
    """
    勾配の計算
    
    Parameters:
    X: 特徴量 (n_samples, n_features)
    y_true: 実際のラベル (n_samples,)
    y_pred: 予測確率 (n_samples, n_classes)
    
    Returns:
    dw: 重みの勾配 (n_classes, n_features)
    db: バイアスの勾配 (n_classes,)
    """
    n_samples = X.shape[0]
    n_classes = y_pred.shape[1]
    
    # ワンホットエンコーディング
    y_one_hot = np.eye(n_classes)[y_true]
    
    # 勾配の計算
    dw = np.dot((y_pred - y_one_hot).T, X) / n_samples
    db = np.mean(y_pred - y_one_hot, axis=0)
    
    return dw, db

# 勾配のテスト
X_test = np.array([[1, 2], [3, 4], [5, 6]])
y_true_test = np.array([0, 1, 2])
y_pred_test = np.array([
    [0.7, 0.2, 0.1],
    [0.1, 0.8, 0.1],
    [0.2, 0.3, 0.5]
])

dw, db = compute_gradients(X_test, y_true_test, y_pred_test)

print("=== 勾配の計算 ===")
print(f"特徴量 X: {X_test}")
print(f"実際のラベル: {y_true_test}")
print(f"予測確率: {y_pred_test}")
print(f"\n重みの勾配 dw: {dw}")
print(f"バイアスの勾配 db: {db}")

# 勾配の可視化
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.imshow(dw, cmap='RdBu', aspect='auto')
plt.colorbar(label='Gradient Value')
plt.title('Weight Gradients (dw)')
plt.xlabel('Features')
plt.ylabel('Classes')

plt.subplot(1, 2, 2)
plt.bar(range(len(db)), db, color='skyblue', alpha=0.7)
plt.xlabel('Classes')
plt.ylabel('Bias Gradient')
plt.title('Bias Gradients (db)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 5. 演習問題

### 演習1: 数値的安定性の確認
異なる実装方法でソフトマックス関数を比較し、数値的安定性の違いを確認してみましょう。

### 演習2: 勾配の検証
数値微分を使用して、計算した勾配が正しいことを確認してみましょう。

### 演習3: 損失関数の最適化
勾配降下法を使用して、損失関数を最小化してみましょう。


## まとめ

このノートブックでは、ソフトマックス関数と多クラス分類の損失関数について詳しく学習しました。

**学習した内容**：
- ソフトマックス関数の実装と数値的安定性
- 交差エントロピー損失の計算
- 勾配の計算方法
- 数値的不安定性への対処

**重要なポイント**：
- ソフトマックス関数は確率の正規化を自動的に行う
- 数値的安定性のため、最大値を引くことが重要
- 交差エントロピー損失は確率のクリップが必要
- 勾配の計算は行列演算で効率的に実行可能

**次のステップ**：
- より高度な最適化手法の学習
- 正則化の実装
- 深層学習への応用
